In [1]:
"""
MILP Example (Pyomo):
Open-or-not decision + shipping quantity

Business:
- You either open a warehouse (y=1) or not (y=0)
- You must satisfy demand
- Shipping can happen ONLY if warehouse is open

Requires:
pip install pyomo
Plus a MILP solver installed (one of these):
- HiGHS (recommended): pip install highspy
- CBC (via conda) or GLPK (often LP-only, not ideal for MILP)
"""

from pyomo.environ import (
    ConcreteModel, Var, Objective, Constraint,
    NonNegativeReals, Binary, minimize, SolverFactory, value
)

def main():
    # ----------------------------
    # 1) Input data (parameters)
    # ----------------------------
    demand = 800          # customer demand units
    ship_cost = 5         # cost per unit shipped
    open_cost = 2000      # fixed cost to open warehouse
    M = 10000             # Big-M: must be >= maximum possible x (>= demand is enough here)

    # ----------------------------
    # 2) Build model
    # ----------------------------
    m = ConcreteModel()

    # Decision variables
    m.x = Var(domain=NonNegativeReals)  # shipped quantity
    m.y = Var(domain=Binary)            # 1=open, 0=closed

    # Objective: minimize total cost
    m.obj = Objective(
        expr=ship_cost * m.x + open_cost * m.y,
        sense=minimize
    )

    # Constraint 1: serve demand
    m.demand = Constraint(expr=m.x >= demand)

    # Constraint 2 (logic): cannot ship unless warehouse is open
    m.logic = Constraint(expr=m.x <= M * m.y)

    # ----------------------------
    # 3) Solve
    # ----------------------------
    # Try HiGHS first (best lightweight option)
    solver = SolverFactory("highs")

    results = solver.solve(m, tee=False)

    # ----------------------------
    # 4) Print results
    # ----------------------------
    print("=== RESULTS ===")
    print("Warehouse open y =", int(round(value(m.y))))
    print("Shipped qty x    =", float(value(m.x)))
    print("Total cost       =", float(value(m.obj)))

    # Extra interpretation (planner-style)
    if value(m.y) < 0.5:
        print("\nInterpretation: warehouse stays closed, so shipping is forced to 0.")
    else:
        print("\nInterpretation: warehouse opens, so shipping is allowed and demand is served.")

if __name__ == "__main__":
    main()


=== RESULTS ===
Warehouse open y = 1
Shipped qty x    = 800.0
Total cost       = 6000.0

Interpretation: warehouse opens, so shipping is allowed and demand is served.
